In [ ]:
# Run by terminal if these modules are not in your enviroment
# pip install pandas-datareader
# pip3 install tensorflow
# pip install numpy
# pip install pandas
# pip install matplotlib
# pip install scipy
# pip install -U scikit-learn
# pip install keras


In [7]:
# Import the Libraries

from web3 import Web3
import pandas as pd
import numpy as np
import pandasql as ps
import string
import re
import math
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [15]:
class ethData:
    
    def ___init__(self, name):
        self.name = name
    
    def blockchainConnection(self, _connectionString):
        if (re.search("(http(|s)\:\/\/)", _connectionString) != None):
            # HTTP Provider
            w3 = Web3(Web3.HTTPProvider(_connectionString))
        elif (re.search("(\.ipc)", _connectionString) != None):
            # IPC Provider
            w3 = Web3(Web3.IPCProvider(connectionString))
        elif (re.search("(ws\:\/\/)", _connectionString) != None):
            # Web Socket Provider
            w3 = Web3(Web3.WebsocketProvider(_connectionString))    
        else:
            w3 = None
        return w3
    
    def getBlockArgs(self, _w3):
        #return list(_w3.eth.get_block('latest', True).keys())
        return ['difficulty', 'extraData', 'gasLimit', 'gasUsed', 'hash', 'logsBloom', 'miner', 'mixHash', 'nonce', 'number', 'parentHash', 'receiptsRoot', 'sha3Uncles', 'size', 'stateRoot', 'timestamp', 'totalDifficulty', 'transactions', 'transactionsRoot', 'uncles']
    def getTransactionArgs(self, _w3):
        #return list(_w3.eth.get_block('latest', True)["transactions"][0].keys())
        return ['blockHash', 'blockNumber', 'from', 'gas', 'gasPrice', 'hash', 'input', 'nonce', 'r', 's', 'to', 'transactionIndex', 'type', 'v', 'value']

    def getTransactionReceiptArgs(self, _w3):
        #return list(_w3.eth.getTransactionReceipt(str(_w3.eth.get_block('latest', True)["transactions"][0]["hash"].hex())).keys())
        return ['blockHash', 'blockNumber', 'contractAddress', 'cumulativeGasUsed', 'from', 'gasUsed', 'logs', 'logsBloom', 'status', 'to', 'transactionHash', 'transactionIndex', 'type']
    
    def getLastBlockNumber(self, _w3):
        return _w3.eth.get_block('latest', True).number;
    
    def addBlockOnDataFrame(self, _block, _df, _columns):
        _df.loc[_df.shape[0]+1] = None
        for key in _columns:
            if self.is_hex(_block[key]):
                _df[key][_df.shape[0]] = str(_block[key].hex())
            else:
                _df[key][_df.shape[0]] = str(_block[key])
    
    def importTransactionList(self, _w3, _block, _blockArgs, _df_transaction, _transactionArgs, _df_transactionReceipt, _transactionReceiptArgs):
        transactions = _block["transactions"]
        for indexTransaction in range(0, len(transactions)):
            #Importing transactions on df_transaction
            transaction = _w3.eth.getTransaction(str(transactions[indexTransaction].hex()))
            self.addBlockOnDataFrame(transaction, _df_transaction, _transactionArgs)
            #Importing transaction receipts on df_transactionReceipt
            transactionReceipt = _w3.eth.getTransactionReceipt(str(transaction["hash"].hex()))
            self.addBlockOnDataFrame(transactionReceipt, _df_transactionReceipt, _transactionReceiptArgs)
            
    def importBlockInterval(self, _w3, _min, _max):
        blockArgs = self.getBlockArgs(_w3)
        transactionArgs = self.getTransactionArgs(_w3)
        transactionReceiptArgs = self.getTransactionReceiptArgs(_w3)
        df_block = pd.DataFrame(columns = blockArgs)
        df_transaction = pd.DataFrame(columns = transactionArgs)
        df_transactionReceipt = pd.DataFrame(columns = transactionReceiptArgs)
        for indexBlock in range(_min, _max):
            block = _w3.eth.get_block(indexBlock)
            #Importing block data
            self.addBlockOnDataFrame(block, df_block, blockArgs)
            #Importing transaction and transaction receipt data
            self.importTransactionList(_w3, block, blockArgs, df_transaction, transactionArgs, df_transactionReceipt, transactionReceiptArgs)  
        return (df_block, df_transaction, df_transactionReceipt)
            
    def is_hex(self, _arg):
        try:
            _arg.hex()
            return True
        except:
            return False

In [16]:
class AI:
    def ___init__(self, name, tables):
        self.name = name
    
    # This function return a dataframe with all stock quotes between two dates
    def getStockQuote(self, _ticker, _provider, _start, _end):
        return web.DataReader(_ticker, data_source = _provider, start = _start, end = _end)

In [17]:
ai = AIDeFiCore()

NameError: name 'AIDeFiCore' is not defined

In [13]:
connectionString = 'https://ropsten.infura.io/v3/9a82d6f712b6474eb712fa507d110055'

In [14]:
w3 = ai.blockchainConnection(connectionString)

In [15]:
w3.isConnected()

True

In [16]:
(df_block, df_transaction, df_transactionReceipt) = ai.importBlockInterval(w3, ai.getLastBlockNumber(w3) - 10, ai.getLastBlockNumber(w3))

In [ ]:
blockArgs = ai.getBlockArgs(w3)
print(blockArgs)

In [ ]:
transactionArgs = ai.getTransactionArgs(w3)
print(transactionArgs)

In [ ]:
transactionReceiptArgs = ai.getTransactionReceiptArgs(w3)
print(transactionReceiptArgs)

In [ ]:
df_block

In [ ]:
df_transaction

In [ ]:
df_transactionReceipt

In [ ]:
w3.eth.get_block(ai.getLastBlockNumber(w3))

In [ ]:
w3.eth.get_transaction('0xc09c3e34691ccb84372a9b892bf30b718c467b4631bf2d7ea94b6b61d99c22fc')

In [ ]:
w3.eth.getTransactionReceipt('0xdf459bc6cd115307826cefca4da04748882a5eb4856fc26961eee0fb178701b0')

In [ ]:
str(w3.eth.get_block('latest', True)["transactions"][0]["hash"].hex())

In [ ]:
list(w3.eth.getTransactionReceipt(str(w3.eth.get_block('latest', True)["transactions"][0]["hash"].hex())).keys())

In [ ]:
block = w3.eth.get_block(10082327, True)
transactions = block["transactions"]
print(transactions)

In [ ]:
len(transactions)

In [ ]:
print(w3.eth.getTransaction(str(transactions[6]["hash"].hex())))